In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

Great! Good to go!


In [2]:
from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model import get_gepnet, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [3]:
from pygraphviz import AGraph
import glob

In [ ]:
# get chromosme from file
graph = [AGraph(g) for g in glob.glob('nb_graphs/rs/run_3/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)
cell_graph.draw_graph(graph, 'nb_graphs/rs/run_3')

In [4]:
# generate new chromosome
# define primitive set
pset = PrimitiveSet('cnn')

# add cellular encoding program symbols
pset.add_program_symbol(cell_graph.end)
pset.add_program_symbol(cell_graph.seq)
pset.add_program_symbol(cell_graph.cpo)
pset.add_program_symbol(cell_graph.cpi)

# add convolutional operations symbols
conv_symbol = convolution.get_symbol()
pset.add_cell_symbol(conv_symbol.conv1x1)
pset.add_cell_symbol(conv_symbol.conv3x3)
pset.add_cell_symbol(conv_symbol.dwconv3x3)
#pset.add_cell_symbol(conv_symbol.conv1x3)
#pset.add_cell_symbol(conv_symbol.conv3x1)
#pset.add_cell_symbol(conv_symbol.maxpool3x3)

def gene_gen():
    return Gene(pset, 3)

ch = Chromosome(gene_gen, 4)
graph, comp_graph = cell_graph.generate_comp_graph(ch)

cell_graph.save_graph(graph, 'nb_graphs/rs/run_4')
cell_graph.draw_graph(graph, 'nb_graphs/rs/run_4')

In [5]:
# import random
# seed = 221
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# enable torch backends
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [10]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=24,
                   repeat_list=[2, 3, 3, 2],
                   classes=45)

net = get_gepnet(conf)
count_parameters(net)

4.604421

In [11]:
path = Path("/home/cliff/NWPU-RESISC45")
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.2) 
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

In [12]:
model_dir = '/home/cliff/ResearchProjects/models/random_search/'
learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=model_dir).mixup()
learn.to_fp16()

Learner(data=ImageDataBunch;

Train: LabelList (20160 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
forest,forest,forest,forest,forest
Path: /home/cliff/NWPU-RESISC45/train;

Valid: LabelList (5040 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
lake,sea_ice,parking_lot,intersection,basketball_court
Path: /home/cliff/NWPU-RESISC45/train;

Test: None, model=GepNet(
  (stem): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
learn.lr_find(end_lr=100)

learn.recorder.plot()

In [ ]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='run_4')
learn.fit_one_cycle(600, 1e-2, wd=0.0004, callbacks=[cb]) 

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,2.696464,2.094940,0.580754,0.419246,01:19
1,2.361426,1.684465,0.489286,0.510714,01:16
2,2.183659,1.520183,0.436706,0.563294,01:17
3,2.028662,1.294214,0.371230,0.628770,01:18
4,1.910251,1.190481,0.349802,0.650198,01:18
5,1.814554,1.052437,0.302183,0.697817,01:18
6,1.724163,0.951455,0.278968,0.721032,01:18
7,1.681807,0.924171,0.266468,0.733532,01:17
8,1.622994,0.846240,0.235516,0.764484,01:18
9,1.571323,0.791677,0.224802,0.775198,01:18


Better model found at epoch 0 with accuracy value: 0.41924601793289185.
Better model found at epoch 1 with accuracy value: 0.5107142925262451.
Better model found at epoch 2 with accuracy value: 0.5632936358451843.
Better model found at epoch 3 with accuracy value: 0.6287698149681091.
Better model found at epoch 4 with accuracy value: 0.6501984000205994.
Better model found at epoch 5 with accuracy value: 0.6978174448013306.
Better model found at epoch 6 with accuracy value: 0.7210317254066467.
Better model found at epoch 7 with accuracy value: 0.7335317730903625.
Better model found at epoch 8 with accuracy value: 0.7644841074943542.
Better model found at epoch 9 with accuracy value: 0.7751984000205994.
Better model found at epoch 11 with accuracy value: 0.8045634627342224.
Better model found at epoch 13 with accuracy value: 0.8277778029441833.
Better model found at epoch 14 with accuracy value: 0.8285714387893677.
Better model found at epoch 15 with accuracy value: 0.8317460417747498.
B

In [ ]:
#####################  Testing  ########################

In [ ]:
graph = [AGraph(g) for g in glob.glob('nb_graphs/rs/run_3/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=16,
                   repeat_list=[2, 2, 3, 1],
                   classes=45)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
tfms = get_transforms(do_flip=False)
path = Path("/home/cliff/NWPU-RESISC45")

bs = 128

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

In [ ]:
model_dir = '/home/cliff/ResearchProjects/models/random_search/'
model = Learner(data, net, metrics=[accuracy, error_rate]).load(model_dir+'run_3')
_, acc, err = model.validate()
print('Accuracy: %.2f | Error: %.2f' %(acc.item()*100, err.item()*100))

In [ ]:
model.validate(data.valid_dl)

In [ ]:
interp = ClassificationInterpretation.from_learner(model)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
#interp.plot_top_losses(9, figsize=(15,15), heatmap=False)

In [ ]:
interp.plot_confusion_matrix(figsize=(15,16), dpi=128)

In [ ]:
#preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
#preds[:5]

In [ ]:
import pickle

In [ ]:
with open('comp_graphs/experiment_1/best/stats.pkl', 'rb') as f:
    stats = pickle.load(f)

In [ ]:
print(stats)